In [ ]:
import pandas as pd
df = pd.read_csv('../datasets/news_dataset.csv')
df.head()

,unique_id,outlet,headline,url,article_text,image,nlp_label,nlp-image_label
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,images/39e6104d56.jpeg,Likely to be Bias,Likely to be Unbiased
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,images/89ce50166b.jpeg,Likely to be Bias,Likely to be Unbiased
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,images/8b320e107e.jpeg,Likely to be Bias,Likely to be Bias
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",images/7536f87654.jpeg,Likely to be Unbiased,Likely to be Unbiased
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",images/c829d1f9a8.jpeg,Likely to be Bias,Likely to be Unbiased


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40945 entries, 0 to 40944
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   unique_id        40945 non-null  object
 1   outlet           40945 non-null  object
 2   headline         40945 non-null  object
 3   url              40945 non-null  object
 4   article_text     40945 non-null  object
 5   image            40945 non-null  object
 6   nlp_label        40945 non-null  object
 7   nlp-image_label  40945 non-null  object
dtypes: object(8)
memory usage: 2.5+ MB


In [7]:
df = df.drop(columns=['image', 'nlp-image_label'])
df.head()

,unique_id,outlet,headline,url,article_text,nlp_label
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,Tij Iginla doesn't shy away from his famous la...,Likely to be Bias
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,This week on The Sunday Magazine with host Piy...,Likely to be Bias
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,An Ontario Conservative MP's use of ChatGPT to...,Likely to be Bias
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,"When it comes to the study of sports, a man's ...",Likely to be Unbiased
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,"To wrap up the 48th season of Quirks & Quarks,...",Likely to be Bias


In [8]:
#lowercase
df['article_text'] = df['article_text'].apply(lambda x: x.lower())

In [9]:
# remove punctuation
import re

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

df['article_text'] = df['article_text'].apply(remove_punctuation)

In [10]:
#encoding for label 

df = pd.get_dummies(df, columns=['nlp_label'], dtype=int)
df.head()

,unique_id,outlet,headline,url,article_text,nlp_label_Likely to be Bias,nlp_label_Likely to be Unbiased
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,tij iginla doesnt shy away from his famous las...,1,0
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,this week on the sunday magazine with host piy...,1,0
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,an ontario conservative mps use of chatgpt to ...,1,0
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,when it comes to the study of sports a mans bo...,0,1
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,to wrap up the 48th season of quirks quarks w...,1,0


In [11]:
#tokenise
import nltk
from nltk.tokenize import word_tokenize

df['article_text_tokens'] = df['article_text'].apply(word_tokenize)

In [12]:
# stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
article_text_tokens_no_stopwords = []
for row in df['article_text_tokens']:
    filtered_row = []
    for w in row:
        if w not in stop_words:
            filtered_row.append(w)
    article_text_tokens_no_stopwords.append(filtered_row)

df['article_text_tokens_no_stopwords'] = article_text_tokens_no_stopwords

In [ ]:
sentiments_df = pd.read_csv('../datasets/generated/news_dataset_sentiment_labels.csv')
# framing_df = pd.read_csv('../datasets/generated/framing_analysis_topics.csv')
topic_df = pd.read_csv('../datasets/generated/articles_with_dominant_topic.csv')

In [ ]:
# Merge the DataFrames on the 'unique_id' column
merged_df = pd.merge(df, sentiments_df[['unique_id', 'negative', 'neutral', 'positive']], on='unique_id', how='left')
# merged_df = pd.merge(merged_df, framing_df[['unique_id', 'topic']], on='unique_id', how='left')
merged_df = pd.merge(merged_df, topic_df[['unique_id', 'dominant_topic']], on='unique_id', how='left')

In [15]:
merged_df.head()

,unique_id,outlet,headline,url,article_text,nlp_label_Likely to be Bias,nlp_label_Likely to be Unbiased,article_text_tokens,article_text_tokens_no_stopwords,negative,neutral,positive
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,tij iginla doesnt shy away from his famous las...,1,0,"[tij, iginla, doesnt, shy, away, from, his, fa...","[tij, iginla, doesnt, shy, away, famous, last,...",0.0,10.0,8.0
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,this week on the sunday magazine with host piy...,1,0,"[this, week, on, the, sunday, magazine, with, ...","[week, sunday, magazine, host, piya, chattopad...",0.0,5.0,6.0
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,an ontario conservative mps use of chatgpt to ...,1,0,"[an, ontario, conservative, mps, use, of, chat...","[ontario, conservative, mps, use, chatgpt, sha...",2.0,18.0,1.0
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,when it comes to the study of sports a mans bo...,0,1,"[when, it, comes, to, the, study, of, sports, ...","[comes, study, sports, mans, body, long, consi...",0.0,11.0,3.0
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,to wrap up the 48th season of quirks quarks w...,1,0,"[to, wrap, up, the, 48th, season, of, quirks, ...","[wrap, 48th, season, quirks, quarks, weve, got...",0.0,14.0,0.0


In [16]:
merged_df_cleaned = merged_df.dropna()

In [17]:
merged_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32705 entries, 0 to 32704
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   unique_id                         32705 non-null  object 
 1   outlet                            32705 non-null  object 
 2   headline                          32705 non-null  object 
 3   url                               32705 non-null  object 
 4   article_text                      32705 non-null  object 
 5   nlp_label_Likely to be Bias       32705 non-null  int32  
 6   nlp_label_Likely to be Unbiased   32705 non-null  int32  
 7   article_text_tokens               32705 non-null  object 
 8   article_text_tokens_no_stopwords  32705 non-null  object 
 9   negative                          32705 non-null  float64
 10  neutral                           32705 non-null  float64
 11  positive                          32705 non-null  float64
dtypes: f

In [18]:
merged_df_cleaned.head()

,unique_id,outlet,headline,url,article_text,nlp_label_Likely to be Bias,nlp_label_Likely to be Unbiased,article_text_tokens,article_text_tokens_no_stopwords,negative,neutral,positive
0,39e6104d56,CBC.ca,"Tij Iginla embraces family name, eager to foll...",https://www.cbc.ca,tij iginla doesnt shy away from his famous las...,1,0,"[tij, iginla, doesnt, shy, away, from, his, fa...","[tij, iginla, doesnt, shy, away, famous, last,...",0.0,10.0,8.0
1,89ce50166b,CBC.ca,"The Sunday Magazine for June 23, 2024 - CBC.ca",https://www.cbc.ca,this week on the sunday magazine with host piy...,1,0,"[this, week, on, the, sunday, magazine, with, ...","[week, sunday, magazine, host, piya, chattopad...",0.0,5.0,6.0
2,8b320e107e,CBC News,"Conservative MP shares inaccurate, ChatGPT-gen...",https://www.cbc.ca,an ontario conservative mps use of chatgpt to ...,1,0,"[an, ontario, conservative, mps, use, of, chat...","[ontario, conservative, mps, use, chatgpt, sha...",2.0,18.0,1.0
3,7536f87654,CBC.ca,"Women's sports are more popular than ever, but...",https://www.cbc.ca,when it comes to the study of sports a mans bo...,0,1,"[when, it, comes, to, the, study, of, sports, ...","[comes, study, sports, mans, body, long, consi...",0.0,11.0,3.0
4,c829d1f9a8,CBC.ca,June 22: Listener Question Show - CBC.ca,https://www.cbc.ca,to wrap up the 48th season of quirks quarks w...,1,0,"[to, wrap, up, the, 48th, season, of, quirks, ...","[wrap, 48th, season, quirks, quarks, weve, got...",0.0,14.0,0.0


In [19]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"  # Use "bert-base-cased" if your data has capitalization
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust num_labels if needed

c:\Users\ZY\anaconda3\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Apply tokenization to the 'article_text_tokens_no_stopwords' column
def tokenize_data(example):
    # Convert the list of tokens to a space-separated string for BERT
    text = " ".join(example)  # Join tokenized words into a single string
    encoding = tokenizer(text, padding=True, truncation=True, max_length=512)
    return encoding

# Apply tokenization to your DataFrame
merged_df_cleaned['input_ids'] = merged_df_cleaned['article_text_tokens_no_stopwords'].apply(lambda x: tokenize_data(x)['input_ids'])
merged_df_cleaned['attention_mask'] = merged_df_cleaned['article_text_tokens_no_stopwords'].apply(lambda x: tokenize_data(x)['attention_mask'])

# If you have labels in your dataset (e.g., 'bias_label' column)
merged_df_cleaned['labels'] = merged_df_cleaned['nlp_label_Likely to be Bias']

C:\Users\ZY\AppData\Local\Temp\ipykernel_20540\2030321308.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_cleaned['input_ids'] = merged_df_cleaned['article_text_tokens_no_stopwords'].apply(lambda x: tokenize_data(x)['input_ids'])
C:\Users\ZY\AppData\Local\Temp\ipykernel_20540\2030321308.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_cleaned['attention_mask'] = merged_df_cleaned['article_text_tokens_no_stopwords'].apply(lambda x: tokenize_data(x)['attention_mask'])
C:\Users\ZY\A

In [21]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset

# Ensure input_ids, attention_mask, and labels are lists
input_ids = merged_df_cleaned['input_ids'].tolist()
attention_mask = merged_df_cleaned['attention_mask'].tolist()
labels = merged_df_cleaned['labels'].astype(int).tolist()  # Convert labels to integers

# Assuming 'input_ids' and 'attention_mask' are lists or numpy arrays
# Ensure that the labels are in a similar format (list/array)

# Train-test split for the first 1000 records
train_texts, val_texts, train_labels, val_labels = train_test_split(input_ids[:1000], labels[:1000], test_size=0.2)

# You should also split the attention masks accordingly
train_attention_mask, val_attention_mask = train_test_split(attention_mask[:1000], test_size=0.2)

# Create a Dataset class to format the data for the model
class TextDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create Dataset objects for train and validation
train_dataset = TextDataset(train_texts, train_attention_mask, train_labels)
val_dataset = TextDataset(val_texts, val_attention_mask, val_labels)

In [22]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

# 3️⃣ Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Save model checkpoints here
    num_train_epochs=3,              # Adjust as needed
    per_device_train_batch_size=8,   # Adjust batch size
    per_device_eval_batch_size=8,    # Batch size for validation
    evaluation_strategy="epoch",     # Evaluate at each epoch
    save_strategy="epoch",           # Save at each epoch
    logging_dir="./logs",            # Logging directory
    logging_steps=100,               # Log every 100 steps
    load_best_model_at_end=True,     # Load best model based on eval loss
    metric_for_best_model="accuracy",
)

# 4️⃣ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,   # Train dataset
    eval_dataset=val_dataset       # Validation dataset
)

# 5️⃣ Start training 🚀
trainer.train()

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
module 'tensorflow' has no attribute 'data'